# 氏名漢字カナ突合モデル

漢字カナ突合モデルはJava17で記述されている。Mavenを用いて漢字カナ突合モデルをビルドする。
仕様等については[README](./README.md)を参照のこと。


## 必要なモジュール

- JDK 17
- Maven 3




## 事前準備
- バージョン
pom.xmlのバージョン番号を書き換える

```xml
    <groupId>jp.go.digital.kanjikana</groupId>
    <artifactId>kanjikana_core_oss</artifactId>
    <version>1.6o</version>   // ここを書き換えるとコンパイル後のJarファイル名にもこのバージョン番号が入る

```

- ファイル
[dict](../dict)で作成した，辞書ファイル及び，[train](../train)で作成したAIモデルのファイルを，Javaのリソースとして使用するために，src/main/resources以下にファイルをコピーする。なお，事前に作成した辞書及び，事前学習済みのモデルを，[resources][https://kktg.digital.go.jp/support/resources/]で配布している。これらのファイルをあらかじめ，../dictおよび，../train/model, ../train/model_r以下にコピーしたのちに，下記スクリプトを実行してもよい。

In [5]:
!mvn clean antrun:run


[INFO] Scanning for projects...
[INFO] 
[INFO] ---------< jp.go.digital.kanjikana:shimei_kanjikana_core_oss >----------
[INFO] Building shimei_kanjikana_core_oss 1.6o
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- clean:3.2.0:clean (default-clean) @ shimei_kanjikana_core_oss ---
[INFO] Deleting /Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/core/target
[INFO] 
[INFO] --- antrun:3.1.0:run (default-cli) @ shimei_kanjikana_core_oss ---
[INFO] Executing tasks
[INFO] Executed tasks
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  1.268 s
[INFO] Finished at: 2025-01-21T15:14:53+09:00
[INFO] ------------------------------------------------------------------------


## コンパイルする


In [6]:
!mvn compile assembly:single


[INFO] Scanning for projects...
[INFO] 
[INFO] ---------< jp.go.digital.kanjikana:shimei_kanjikana_core_oss >----------
[INFO] Building shimei_kanjikana_core_oss 1.6o
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- resources:3.2.0:resources (default-resources) @ shimei_kanjikana_core_oss ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] Using 'UTF-8' encoding to copy filtered properties files.
[INFO] Copying 28 resources
[INFO] The encoding used to copy filtered properties files have not been set. This means that the same encoding will be used to copy filtered properties files as when copying other filtered resources. This might not be what you want! Run your build with --debug to see which files might be affected. Read more at https://maven.apache.org/plugins/maven-resources-plugin/examples/filtering-properties-files.html
[INFO] 
[INFO] --- compiler:3.13.0:compile (default-compile) @ shimei_kan

## 実行する

漢字姓名とカナ姓名が入力されたファイルをCSVもしくはTSV形式で用意する。漢字姓名とカナ姓名以外のフィールドが含まれていても良い。
なお，VMパラメタとして`-Xmx4096M`を指定し，４Gのメモリを確保すること。
### 実行クラス
jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchMain
### パラメタ
- infile
入力ファイル。行に漢字姓名とカナ姓名を含むファイルで，TSVもしくはCSV形式で作成しること。また，ヘッダがあっても良くヘッダがある場合には`has_header`パラメタを指定する。また，漢字姓名とカナ姓名の列を`kanji_idx`と`kana_idx`パラメタで指定する
- okfile
漢字姓名とカナ姓名が一致と判断されたレコードが保存されるファイル
- ngfile
漢字姓名とカナ姓名が不一致と判断されたレコードが保存されるファイル
- logfile
入力ファイルの各行を実行する際のログファイル
- kanji_idx
infileのファイルの漢字姓名の列番号，0から始まる
- kana_idx
infileのファイルのカナ姓名の列番号，0から始まる
- sep
infileの各列の区切り文字を指定する。csvもしくはtsvを指定する。
- thread_num
infileに大量の行が含まれる場合には，本プログラムをスレッドで多重化して計算することができる。`thread_num`に1を指定すると（デフォルト），シングルスレッドで実行され，それ以上を値を指定するとその数だけスレッドで計算される。スレッドでケインさんした場合には，`okfile`と`ngfile`，`logfile`の末尾にスレッド番号が付与されたファイルが作成される
- has_header
infileにヘッダ業がある場合には`true`を指定する。デフォルトは`false`であり，infileにヘッダ行は含まれない
- strategy
漢字姓名とカナ姓名の突合モデルをどれを使うかを指定する。BASIC, ONLY_AI, ONLY_DICT, AI, ENSEMBLEから選択し，セットする。    

  |モデル|内容|
  |-----|---|
  |ENSEMBLE||
  |BASIC|姓名辞書，異体字辞書との突合のみ|
  |ONLY_AI|AIモデルとの突合のみ|
  |ONLY_DICT|姓名辞書及び単漢字辞書，異体字辞書との突合のみ|
  |AI|BASIC + ONLY_AI|
  

In [13]:
# 出力結果がoutput.txtに出力される
!java -Xmx4096M -Dlog4j.configurationFile=log4j2.xml -classpath target/shimei_kanjikana_core_oss-1.6o-jar-with-dependencies.jar jp.go.digital.kanjikana.core.executor.match.KanjiKanaMatchMain --infile input.txt --outfile outfile.txt --kanji_idx 1 --kana_idx 2 --strategy ENSEMBLE --has_header true --thread_num 1 --sep csv


15:26:17.148 [main] DEBUG ai.djl.repository.zoo.DefaultModelZoo -- Scanning models in repo: class ai.djl.repository.SimpleRepository, /var/folders/nd/8rmfsv815m76kpkw1wjlgddc0000gn/T/model_10219695931157234501.tmp
15:26:17.151 [main] DEBUG ai.djl.engine.Engine -- Registering EngineProvider: PyTorch
15:26:17.151 [main] DEBUG ai.djl.engine.Engine -- Found default engine: PyTorch
15:26:17.162 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Loading model with Criteria:
	Application: UNDEFINED
	Input: class ai.djl.ndarray.NDList
	Output: class ai.djl.ndarray.NDList
	Engine: PyTorch
	ModelZoo: ai.djl.localmodelzoo
	No translator supplied

15:26:17.162 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Searching model in specified model zoo: ai.djl.localmodelzoo
15:26:17.162 [main] DEBUG ai.djl.repository.zoo.ModelZoo -- Checking ModelLoader: ai.djl.localmodelzoo:model_10219695931157234501.tmp UNDEFINED [
	ai.djl.localmodelzoo/model_10219695931157234501.tmp/model_10219695931157234501.tmp {}
]
15:2

## 結果を表示する

In [14]:
!head outfile.txt

no,kanji,kana,okng,result_flg,result_per,result,start_date,end_date
1,山田　太郎,ヤマダ　ハナコ,ng,false,0,{true;山田;ヤマダ;WordEngine;DictOSS;0:/canna/freewnn/kakasi/;null}{false;太郎;ハナコ;WordEngine;;0:/;}[<|IDictCharModel|{true;山田;ヤマダ;WordEngine;DictOSS;0:/canna/freewnn/kakasi/;null}{false;太郎;ハナコ;CharEngine;;0:/;}><|AiCharModel|{true;山田;ヤマダ;AiWordEngine;;0:/;rank:0;Probability:0.693487004581911}{false;太郎;ハナコ;AiCharEngine;;0:/;}>],2025-01-21 15:26:16.477 +0900,2025-01-21 15:26:20.107 +0900
2,山田　太郎,ヤマダタロウ,ok,true,80,{false;山田　太郎;ヤマダタロウ;WordEngine;;0:/;}[<|DictCharModel|{true;山田;ヤマダ;CharEngine;DictSeimei;0:/wikipedia/;null}{true;太郎;タロウ;CharEngine;DictSeimei;0:/wikipedia/;null}><|AiCharModel|{true;山田太郎;ヤマダタロウ;AiCharEngine;;0:/;rank:0;Probability:0.9998961699101548}>],2025-01-21 15:26:20.109 +0900,2025-01-21 15:26:20.881 +0900
3,山田　太郎,サンダ　フトロウ,ok,false,30,{false;山田;サンダ;WordEngine;;0:/;}{false;太郎;フトロウ;WordEngine;;0:/;}[<|DictCharModel|{true;山;サン;CharEngine;DictSeimei;0:/wikipedia/;null}{true;田;ダ;CharEngine;

## javadocを作成する
Javadocを作成し，/docs以下へ移動する。GithubPagesで見えるようにする。 

URLは [https://studious-chainsaw-5k2p5q6.pages.github.io/](https://studious-chainsaw-5k2p5q6.pages.github.io/)


In [15]:
!mvn javadoc:javadoc

[INFO] Scanning for projects...
[INFO] 
[INFO] ---------< jp.go.digital.kanjikana:shimei_kanjikana_core_oss >----------
[INFO] Building shimei_kanjikana_core_oss 1.6o
[INFO]   from pom.xml
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] >>> javadoc:3.11.2:javadoc (default-cli) > generate-sources @ shimei_kanjikana_core_oss >>>
[INFO] 
[INFO] <<< javadoc:3.11.2:javadoc (default-cli) < generate-sources @ shimei_kanjikana_core_oss <<<
[INFO] 
[INFO] 
[INFO] --- javadoc:3.11.2:javadoc (default-cli) @ shimei_kanjikana_core_oss ---
[INFO] Skipping javadoc generation, everything is up to date.
[INFO] ------------------------------------------------------------------------
[INFO] BUILD SUCCESS
[INFO] ------------------------------------------------------------------------
[INFO] Total time:  1.359 s
[INFO] Finished at: 2025-01-21T15:27:06+09:00
[INFO] ------------------------------------------------------------------------
